In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys

# reference main directory in existing folder
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import pandas as pd 
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from src.paths import (RAW_DATA_DIR, CLEANED_DATA_DIR, SUGGESTED_CHANGES_DIR, MEAN_CATEGORIES_DIR, 
                       PLAYER_CATEGORIES_DIR, STREAMLIT_DIR)

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import pandas as pd
from scipy.spatial.distance import cosine

# show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
# display the max column width
pd.set_option('display.max_colwidth', None)


In [26]:
# Load All Means
elite_mean_df = pd.read_csv(MEAN_CATEGORIES_DIR / 'elite_and_strong_mean.csv')
suboptimal_mean_df = pd.read_csv(MEAN_CATEGORIES_DIR / 'suboptimal_mean.csv')
elite_mean_by_role = pd.read_csv(MEAN_CATEGORIES_DIR / 'elite_means_by_role.csv')
suboptimal_mean_by_role = pd.read_csv(MEAN_CATEGORIES_DIR / 'suboptimal_means_by_role.csv')

# Load Suboptimal Players
suboptimal_players = pd.read_csv(PLAYER_CATEGORIES_DIR / 'suboptimal_players.csv')
elite_and_strong_players_df = pd.read_csv(PLAYER_CATEGORIES_DIR / 'elite_and_strong_players.csv')

# Load Suggested Data
suggested_roles_df = pd.read_csv(SUGGESTED_CHANGES_DIR / 'suggested_changes.csv')

In [27]:
from src.const import *

# rename Unnamed: 0 to Stat
elite_mean_by_role.rename(columns={'Unnamed: 0': 'Stat'}, inplace=True)
suboptimal_mean_by_role.rename(columns={'Unnamed: 0': 'Stat'}, inplace=True)


In [28]:
elite_mean_df.head()

,classified_role,G,GS,IP,TBF,W,L,CG,ShO,SV,BS,HLD,SD,MD,Pulls,ERA,R,ER,H,HR,SO,BB,IBB,HBP,WP,BK,Events,GB,LD,FB,IFFB,BU,IFH,BUH,Balls,Strikes,Pitches,RS,RS_per_9,K_pct,BB_pct,K_minus_BB_pct,K_per_9,BB_per_9,K_to_BB,H_per_9,HR_per_9,AVG,WHIP,BABIP,LOB_pct,GB_pct,LD_pct,FB_pct,IFFB_pct,GB_to_FB,HR_to_FB,IFH_pct,BUH_pct,FIP,xFIP,SIERA,tERA,kwERA,E_minus_F,RAR,WAR,Dollars,WPA,RE24,REW,pLI,inLI,gmLI,exLI,WPA_to_LI,Clutch,OSwing_pct,ZSwing_pct,Swing_pct,OContact_pct,ZContact_pct,Contact_pct,Zone_pct,FStrike_pct,SwStr_pct,CStr_pct,CSW_pct,ERA_minus,FIP_minus,xFIP_minus,Pace,RA9_WAR,BIP_Wins,LOB_Wins,FDP_Wins,Pull_pct,Cent_pct,Oppo_pct,Soft_pct,Med_pct,Hard_pct,TTO_pct,FRM,K_pct_plus,BB_pct_plus,K_per_9_plus,BB_per_9_plus,K_to_BB_plus,H_per_9_plus,HR_per_9_plus,AVG_plus,WHIP_plus,BABIP_plus,LOB_pct_plus,GB_pct_plus,LD_pct_plus,FB_pct_plus,HR_to_FB_pct_plus,Pull_pct_plus,Cent_pct_plus,Oppo_pct_plus,Med_pct_plus,Hard_pct_plus,EV,LA,Barrels,Barrel_pct,maxEV,HardHit,HardHit_pct,FA_pct,FAv,wFB,wFB_per_c,FA_pct_sc,vFA_sc,FA_X_sc,FA_Z_sc,wFA_sc,wFA_per_c_sc,OSwing_pct_sc,ZSwing_pct_sc,Swing_pct_sc,OContact_pct_sc,ZContact_pct_sc,Contact_pct_sc,Zone_pct_sc,botERA,botOvr_FA,botStf_FA,botCmd_FA,botOvr,botStf,botCmd,botxRV100,Stf_plus_FA,Loc_plus_FA,Pit_plus_FA,Stuff_plus,Location_plus,Pitching_plus,average_batters_faced_per_appearance_SP,average_innings_pitched_per_appearance_SP,average_batters_faced_per_appearance_RP,average_innings_pitched_per_appearance_RP,z_total_score
0,Middle Reliever,19.969697,5.492424,46.865909,198.818182,3.037879,2.575758,0.022727,0.007576,0.848485,0.621212,2.681818,4.674242,2.462121,16.333333,4.072056,22.553030,21.045455,43.189394,5.848485,45.439394,16.734848,0.500000,2.219697,1.560606,0.196970,134.424242,58.265152,26.257576,48.803030,4.871212,0.992424,3.522727,0.333333,280.734848,495.681818,776.416667,27.151515,5.190210,0.229958,0.089537,0.140419,8.708535,3.474662,2.999405,8.137700,1.121883,0.232446,1.290262,0.282607,0.736832,0.425883,0.196395,0.377722,0.101063,1.280071,0.117068,0.052678,0.178012,4.242375,4.366265,4.240895,4.311998,4.533750,-0.170319,5.605694,0.577028,4.616223,0.198193,2.336445,0.228879,0.910361,0.844379,0.942256,1.057915,0.178231,0.021999,0.315052,0.686692,0.471601,0.631897,0.853617,0.767718,0.420703,0.612597,0.109937,0.165835,0.275772,93.935250,97.904657,100.803539,18.701857,0.657837,0.058666,0.022143,0.080809,0.400070,0.350581,0.249344,0.160343,0.516370,0.323288,0.348441,0.048917,101.155249,104.183282,99.624888,105.114046,112.046800,95.401578,91.516244,94.592398,98.109685,95.749409,102.504866,100.234408,98.227708,100.828543,91.915087,97.358800,101.421616,102.471307,99.600457,99.050459,88.794928,12.739219,10.098485,0.075299,110.747573,51.954545,0.386527,0.490708,93.789252,1.191407,0.046898,0.316199,93.804475,-3.049857,8.538348,0.341301,-0.492114,0.280705,0.656249,0.471739,0.567139,0.849067,0.766670,0.508058,4.352644,49.043690,47.193565,50.038939,51.801722,49.641852,52.340329,0.014855,91.620171,100.295878,95.513711,97.322259,100.039616,99.266676,6.148899,3.141829,7.092888,2.593418,14.951337
1,Setup/Closer Pitcher,33.696000,2.444000,44.592000,187.600000,2.744000,2.420000,0.012000,0.008000,3.452000,1.632000,5.728000,10.272000,4.944000,23.988000,3.814553,20.264000,18.332000,38.608000,5.120000,47.232000,16.256000,0.688000,2.168000,1.676000,0.240000,121.944000,52.880000,23.340000,44.344000,4.744000,1.296000,3.436000,0.376000,263.844000,475.616000,739.460000,21.536000,4.282318,0.248993,0.091403,0.157588,9.451366,3.511849,3.190222,7.866813,0.994359,0.226344,1.264296,0.283817,0.740574,0.437162,0.191322,0.371516,0.105187,1.366928,0.112234,0.068656,0.159067,3.920891,4.140936,3.932052,3.926347,4.327702,-0.106338,5.502175,0.555641,4.445128,0.375350,3.526146,0.354196,1.068079,0.971472,1.077630,1.194561,0.309619,-0.065922,0.323889,0.676734,0.472670,0.602716,0.842229,0.746558,0.420827,0.617146,0.119266,0.167445,0.286708,88.226535,90.645950,95.580623,18.970364,0.618593,0.083251,-0.020299,0.062952,0.400342,0.344868,

In [29]:
# for subotptimal players attach the column 'Suggested Role' and similarity score
suboptimal_players = suboptimal_players.merge(suggested_roles_df[['PlayerId', 'Suggested Role', 'Similarity Score', 'Positive Stats Within One Std Dev of Elite Mean',
       'Negative Stats Within One Std Dev of Elite Mean']], on='PlayerId')

In [30]:
# select chosen players, Daniel Lynch IV, Chase Silseth, and Jovani Moran
chosen_players = suboptimal_players[suboptimal_players['Name'].isin(['Daniel Lynch IV', 'Chase Silseth', 'Jovani Moran'])]

# drop the duplicates
chosen_players = chosen_players.drop_duplicates(subset='Name')

# rename classified_role to 'Current Role'
chosen_players.rename(columns={'classified_role': 'Current Role'}, inplace=True)

# Save to Streamlit Dir
chosen_players.to_csv(STREAMLIT_DIR / 'chosen_players.csv', index=False)

In [17]:
import re

def parse_stats_column(stats_str):
    """
    Parses a string of stats and z-scores and returns a list of tuples with metric names and z-scores.
    
    :param stats_str: String containing stats and z-scores in the format 'metric (z-score: value)'
    :return: List of tuples [(metric1, z_score1), (metric2, z_score2), ...]
    """
    # Regular expression pattern to match the metric names and z-scores
    pattern = r'(\w+)\s\(z-score:\s([-\d.]+)\)'
    return re.findall(pattern, stats_str)

In [15]:
import plotly.graph_objects as go

def visualize_stats(stats_str, player_name):
    """
    Visualizes the stats and z-scores using a bar chart.
    
    :param stats_str: String containing stats and z-scores in the format 'metric (z-score: value)'
    :param player_name: Name of the player to be used in the chart title
    """
    # Parse the stats string to get metrics and their z-scores
    stats = parse_stats_column(stats_str)
    metrics, z_scores = zip(*stats)  # Unpack the list of tuples
    z_scores = [float(z) for z in z_scores]  # Convert string z-scores to float

    # Create the bar chart
    fig = go.Figure(data=[
        go.Bar(
            x=metrics,
            y=z_scores,
            marker_color=['blue' if z >= 0 else 'red' for z in z_scores]
        )
    ])

    # Update chart layout
    fig.update_layout(
        title=f'{player_name} Stats Within One Std Dev of Elite Mean',
        xaxis_title='Metrics',
        yaxis_title='Z-Score',
        yaxis=dict(range=[min(z_scores) - 0.1, max(z_scores) + 0.1])  # Set y-axis range
    )

    # Display the figure
    fig.show()

In [18]:
# Example stats string from the screenshot
stats_str = chosen_players.iloc[0]['Stats Within One Std Dev of Elite Mean']

# Call the visualization function for a player
visualize_stats(stats_str, "Silsseth")

In [ ]:

"""   #st.write(f"**Explanation:** {row['Explanation']}")
    st.write(f"**Positive Stats Within One Std Dev of the Suggested Roles Elite Mean:** {row['Positive Stats Within One Std Dev of Elite Mean']}")
    st.write(f"**Negative Stats Within One Std Dev of Suggested Roles Elite Mean** {row['Negative Stats Within One Std Dev of Elite Mean']}")

    st.write('Metric Selection')
    # Create a dropdown for metric selection
    metric_dict_options = combined_metrics_2
    unique_key = f"metric_select_{row['PlayerId']}"
    selected_metric = st.selectbox('Select a metric:', list(metric_dict_options.keys()), key=unique_key)
    st.write(f"You selected: {selected_metric}")
    """